In [55]:
from easygui import fileopenbox
from librosa import load
import os
import winsound

## Selecting the audio file manually

---

In [61]:
default_path_to_dataset = os.path.abspath(os.path.join("Datasets","RAVDESS_DATASET"))
choosen_path = fileopenbox(msg="Choose a voice sample",title="Test Audio Selector",default=default_path_to_dataset,filetypes="*.wav")

## Play The Selected Audio

In [63]:
winsound.PlaySound(choosen_path,winsound.SND_FILENAME)

# Extract The Original Emotion Details from File Name
---

In [56]:
emotion_dictionary = {
        1:'neutral',
        2:'calm',
        3:'happy',
        4:'sad',
        5:'angry',
        6:'fearful',
        7:'disgust',
        8:'surprised'
    }

In [64]:
def find_emotion(filename):
    """
        @filename : taking the voice input filename 
        
        @returning emotion code by extracting it 
    """
    parta,_ = filename.split('.')
    codes = parta.split('-')
    #print(*codes)
    return int(codes[2])

In [62]:
file_name = choosen_path.split("\\")[-1]
original_emotion = find_emotion(file_name)
print(emotion_dictionary[original_emotion])

surprised


# Extracting The Features of the selected Audio 
-----